### Importing libraries

In [1]:
import requests
import json
import pandas as pd
import urllib.parse
from pybliometrics.scopus import AffiliationRetrieval

In [2]:
#Define your API key here
MY_API_KEY='6545798661bccd5ab7ef446593a92c2f'

In [79]:
resp = requests.get("https://api.elsevier.com/content/search/scopus?query=( TITLE ( sars-cov-2 )  OR  TITLE ( covid-19 ) )  AND  ( ABS ( testing )  OR  ABS ( test )  OR  ABS ( assay )  OR  ABS ( sample )  OR  ABS ( specimen )  OR  ABS ( swab )  OR  ABS ( molecular )  OR  ABS ( antigen )  OR  ABS ( antibody )  OR  ABS ( serology )  OR  ABS ( pcr )  OR  TITLE ( poc )  OR  ABS ( diagnostic )  OR  ABS ( {limit of detection} )  OR  ABS ( lamp )  OR  ABS ( crispr )  OR  ABS ( {spike protein} )  OR  ABS ( reagent )  OR  ABS ( {nasal swab} )  OR  ABS ( nasopharyngeal )  OR  ABS ( oropharyngeal )  OR  ABS ( pooling )  OR  ABS ( screening ) )  AND  PUBYEAR  >  2018  AND  DOCTYPE(ar) OR DOCTYPE(re)  AND  LANGUAGE(english)",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
#,'X-ELS-Insttoken':'c2eb041217badb7ae8addef94d71947e'
print(resp.status_code)#Should be 200
print (json.dumps(resp.json(),
                 sort_keys=True,
                 indent=4, separators=(',', ': ')))#Should populate some results

200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "affiliation": [
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "The First Affiliated Hospital, Zhejiang University School of Medicine"
                    },
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "Zhejiang University"
                    },
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "Key Laboratory of Clinical In Vitro Diagnostic Techniques of Zhejiang Province"
                  

In [5]:
resp = requests.get("https://api.elsevier.com/content/search/scopus?query=( TITLE ( Serology characteristics of SARS-CoV-2 infection since exposure and post symptom onset))",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
#,'X-ELS-Insttoken':'c2eb041217badb7ae8addef94d71947e'
print(resp.status_code)#Should be 200
print (json.dumps(resp.json(),
                 sort_keys=True,
                 indent=4, separators=(',', ': ')))#Should populate some results

200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "affiliation": [
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "Zhejiang University School of Medicine"
                    },
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "Zhejiang University"
                    },
                    {
                        "@_fa": "true",
                        "affiliation-city": "Hangzhou",
                        "affiliation-country": "China",
                        "affilname": "Key Laboratory of Clinical In Vitro Diagnostic Techniques of Zhejiang Province"
                    }
                ],
        

### Defining the columns of a Dataframe that are required in our result

In [114]:
columns = ['eid','Affliation-name','affiliation-city','affiliation-country','affiliation-afid','affiliation-url','affiliation-org-url']
df = pd.DataFrame(columns=columns,index=range(1000001))
#147073

## should go upto 147073*5 times atleast = 735365 ~ 1000001

In [115]:
df.columns

Index(['eid', 'Affliation-name', 'affiliation-city', 'affiliation-country',
       'affiliation-afid', 'affiliation-url', 'affiliation-org-url'],
      dtype='object')

### Result generation - Remember to use the working URL in the code below for the variable 'url'. It should also contain your API key and include insttoken if you've one

In [116]:
count = 0
offset = 0
original_count = 0
cursor = "*"
#147072
while original_count<147072:
    print(original_count,offset, count)
    url = "https://api.elsevier.com/content/search/scopus?query=( TITLE ( sars-cov-2 )  OR  TITLE ( covid-19 ) )  AND  ( ABS ( testing )  OR  ABS ( test )  OR  ABS ( assay )  OR  ABS ( sample )  OR  ABS ( specimen )  OR  ABS ( swab )  OR  ABS ( molecular )  OR  ABS ( antigen )  OR  ABS ( antibody )  OR  ABS ( serology )  OR  ABS ( pcr )  OR  TITLE ( poc )  OR  ABS ( diagnostic )  OR  ABS ( {limit of detection} )  OR  ABS ( lamp )  OR  ABS ( crispr )  OR  ABS ( {spike protein} )  OR  ABS ( reagent )  OR  ABS ( {nasal swab} )  OR  ABS ( nasopharyngeal )  OR  ABS ( oropharyngeal )  OR  ABS ( pooling )  OR  ABS ( screening ) )  AND  PUBYEAR  >  2018  AND  DOCTYPE(ar) OR DOCTYPE(re)  AND  LANGUAGE(english)&view=COMPLETE&cursor="+cursor+"&view=REF"
    resp = requests.get(url,headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
    #,'X-ELS-Insttoken':'c2eb041217badb7ae8addef94d71947e'
    
    results = resp.json()
    entries = results['search-results']['entry']
    cur = results['search-results']['cursor']['@next']
    cursor = urllib.parse.quote(cur)
    print(cursor)
    original_count +=25
    for i in range(len(entries)):
#         df.loc[count]['eid'] = entries[i]['eid'] if 'eid' in entries[i] else None
#         print(count)

        if 'affiliation' in entries[i]:
            for j in range(len( entries[i]['affiliation'])):
                df.loc[count]['eid'] = entries[i]['eid'] if 'eid' in entries[i] else None
            
                if 'affilname' in entries[i]['affiliation'][j] and entries[i]['affiliation'][j]['affilname'] is not None:
                    df.loc[count]['Affliation-name'] = entries[i]['affiliation'][j]['affilname']
                else:
                    df.loc[count]['Affliation-name'] = None
                    
                if 'affiliation-city' in entries[i]['affiliation'][j] and entries[i]['affiliation'][j]['affiliation-city'] is not None:
                    df.loc[count]['affiliation-city'] = entries[i]['affiliation'][j]['affiliation-city']
                else:
                    df.loc[count]['affiliation-city'] = None                    
                    
                if 'affiliation-country' in entries[i]['affiliation'][j] and entries[i]['affiliation'][j]['affiliation-country'] is not None:
                    df.loc[count]['affiliation-country'] = entries[i]['affiliation'][j]['affiliation-country']
                else:
                    df.loc[count]['affiliation-country'] = None
                
                if 'afid' in entries[i]['affiliation'][j] and entries[i]['affiliation'][j]['afid'] is not None:
                    df.loc[count]['affiliation-afid'] = entries[i]['affiliation'][j]['afid']
#                     aff = AffiliationRetrieval(entries[i]['affiliation'][j]['afid'])
#                     df.loc[count]['affiliation-org-url'] = aff.org_URL
                else:
                    df.loc[count]['affiliation-afid'] = None
                
                if 'affiliation-url' in entries[i]['affiliation'][j] and entries[i]['affiliation'][j]['affiliation-url'] is not None:
                    df.loc[count]['affiliation-url'] = entries[i]['affiliation'][j]['affiliation-url']
                else:
                    df.loc[count]['affiliation-url'] = None
                                
                count+=1
        else:
            df.loc[count]['eid'] = entries[i]['eid'] if 'eid' in entries[i] else None
            df.loc[count]['Affliation-name'] = None
            df.loc[count]['affiliation-city'] = None
            df.loc[count]['affiliation-country'] = None
            df.loc[count]['affiliation-afid'] = None
            df.loc[count]['affiliation-url'] = None
            df.loc[count]['affiliation-org-url'] = None
#         count+=1
    offset = count

0 0 0
AoJRwpZNMjItczIuMC04NTE2MTI3NTEyOA%3D%3D
25 68 68
AoJRwpZNMjItczIuMC04NTE2MDk2MjQ1NQ%3D%3D
50 155 155
AoJRwpZNMjItczIuMC04NTE2MDQ5MTI1MQ%3D%3D
75 237 237
AoJRwpZNMjItczIuMC04NTE2MDIxODY4MA%3D%3D
100 328 328
AoJRwpZNMjItczIuMC04NTE1OTk0ODk1Ng%3D%3D
125 440 440
AoJRwpZNMjItczIuMC04NTE1OTcyMzE1MQ%3D%3D
150 554 554
AoJRwpZNMjItczIuMC04NTE1OTM5ODI1OA%3D%3D
175 648 648
AoJRwpZNMjItczIuMC04NTE1ODg2NjExMQ%3D%3D
200 794 794
AoJRwpZNMjItczIuMC04NTE1ODA1MTc5Mg%3D%3D
225 871 871
AoJRwpZNMjItczIuMC04NTE1NDU1MjEwMw%3D%3D
250 970 970
AoJRwpZNMjItczIuMC04NTE1MzU1MzA1NA%3D%3D
275 1071 1071
AoJRwpZNMjItczIuMC04NTE1Mjk3ODc0Mw%3D%3D
300 1176 1176


KeyboardInterrupt: 

In [117]:
len(df['affiliation-city'].unique())

462

In [118]:
df.isnull().sum()

eid                     998825
Affliation-name         998825
affiliation-city        998828
affiliation-country     998825
affiliation-afid        998825
affiliation-url         998825
affiliation-org-url    1000001
dtype: int64

In [119]:
df

,eid,Affliation-name,affiliation-city,affiliation-country,affiliation-afid,affiliation-url,affiliation-org-url
0,2-s2.0-85148333013,"The First Affiliated Hospital, Zhejiang Univer...",Hangzhou,China,60118428,https://api.elsevier.com/content/affiliation/a...,NaN
1,2-s2.0-85148333013,Zhejiang University,Hangzhou,China,60003970,https://api.elsevier.com/content/affiliation/a...,NaN
2,2-s2.0-85148333013,Key Laboratory of Clinical In Vitro Diagnostic...,Hangzhou,China,125028487,https://api.elsevier.com/content/affiliation/a...,NaN
3,2-s2.0-85147796186,Zhengzhou University,Zhengzhou,China,60018554,https://api.elsevier.com/content/affiliation/a...,NaN
4,2-s2.0-85147796186,Hong Kong Baptist University,None,Hong Kong,60014347,https://api.elsevier.com/content/affiliation/a...,NaN
...,...,...,...,...,...,...,...
999996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df["affiliation-url"].iloc[9000]

nan

In [91]:
df.to_excel('affiliation_cols_with_org_urls_part1.xlsx',index=False)